In [1]:
!pip install openai==1.10.0 tiktoken==0.5.2 ragas==0.0.22 langchain-openai==0.0.6


INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 246.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 25.2 MB/s e

In [2]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "my key"

llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.1 MB/s eta 0:00:00


In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")

pages = loader.load_and_split()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

datat_docs = text_splitter.split_documents(pages)

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embed_model = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
import openai
import requests

# 创建自定义错误类
class OpenAIError:
    class Timeout(Exception):
        pass

    class APIError(Exception):
        pass

    class APIConnectionError(Exception):
        pass

# 将自定义错误类添加到 openai 模块
openai.error = OpenAIError

# 示例文档列表
docs = datat_docs  # 您的文档列表

embed_model = OpenAIEmbeddings()

# 使用 tenacity 创建重试装饰器
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

def create_retry_decorator():
    return retry(
        reraise=True,
        stop=stop_after_attempt(5),
        wait=wait_exponential(min=1, max=60),
        retry=(retry_if_exception_type(openai.error.Timeout)
               | retry_if_exception_type(openai.error.APIError)
               | retry_if_exception_type(openai.error.APIConnectionError))
    )

retry_decorator = create_retry_decorator()

@retry_decorator
def create_vectorstore(docs, embed_model):
    # 捕获标准异常并转换为自定义异常
    try:
        return Chroma.from_documents(documents=docs, embedding=embed_model, collection_name="openai_embed")
    except requests.exceptions.Timeout as e:
        raise openai.error.Timeout from e
    except requests.exceptions.RequestException as e:
        raise openai.error.APIError from e
    except requests.exceptions.ConnectionError as e:
        raise openai.error.APIConnectionError from e

try:
    vectorstore = create_vectorstore(docs, embed_model)
    print("Vector store created successfully.")
except openai.error.Timeout:
    print("Request timed out.")
except openai.error.APIError:
    print("API returned an error response.")
except openai.error.APIConnectionError:
    print("Failed to connect to the API server.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


An unexpected error occurred: Could not import chromadb python package. Please install it with `pip install chromadb`.


In [8]:
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 8.2 MB/s eta 0:00:00
 

In [9]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embed_model , collection_name="openai_embed")

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
)
question = "In which areas does Baichuan 2 outperform other open-source models?"
result = qa_chain({"query": question})


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [11]:
result

{'query': 'In which areas does Baichuan 2 outperform other open-source models?',
 'result': 'Baichuan 2 outperforms other open-source models in areas such as math and code problems, medical and legal domain tasks, as well as in specific benchmarks like MedQA and JEC-QA. Additionally, Baichuan 2-13B-Base shows improvement in tasks such as zh-ar, zh-ru, and zh-ja when compared to other models.',
 'source_documents': [Document(page_content='overall performance of the Baichuan 2 base models\ncompared to other open or closed-sourced models', metadata={'page': 1, 'source': 'https://arxiv.org/pdf/2309.10305.pdf'}),
  Document(page_content='With such a massive amount of training data,\nBaichuan 2 achieves significant improvements over\nBaichuan 1. On general benchmarks like MMLU\n(Hendrycks et al., 2021a), CMMLU (Li et al.,\n2023), and C-Eval (Huang et al., 2023), Baichuan\n2-7B achieves nearly 30% higher performance\ncompared to Baichuan 1-7B. Specifically, Baichuan\n2 is optimized to improve

问题及答案

1.What are the two main configurations of Baichuan 2 models?

Baichuan 2 models come in two main configurations: 7 billion parameters and 13 billion parameters.

2.How many tokens were used to train the Baichuan 2 models?

The models were trained on 2.6 trillion tokens.

3.In which areas does Baichuan 2 outperform other open-source models?

Baichuan 2 outperforms in math, code, medical, and legal tasks.

4.What is the purpose of the chat models included in Baichuan 2?

The chat models are optimized for human interaction.

5.How does Baichuan 2 contribute to the AI community?

Baichuan 2's models and training checkpoints are open-sourced to support further research and development.

6.What is the vocabulary size of Baichuan 2?

The vocabulary size is 100,000 tokens.

7.What benchmarks were used to evaluate Baichuan 2's performance?

Specific benchmarks were not detailed in the summary, but it was mentioned that Baichuan 2 performs well across various tasks.

8.What are some specific tasks Baichuan 2 excels in?

Baichuan 2 excels in tasks related to math, code, medical, and legal fields.

9.What is the significance of open-sourcing Baichuan 2's models and training checkpoints?

Open-sourcing allows the AI community to use, study, and build upon Baichuan 2, fostering further innovation and research.

10.How does Baichuan 2 handle multilingual tasks?

Baichuan 2 is designed as a multilingual language model, capable of performing tasks in multiple languages.

In [ ]:
from ragas.langchain.evalchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# create evaluation chains
faithfulness_chain = RagasEvaluatorChain(metric=faithfulness)
answer_rel_chain = RagasEvaluatorChain(metric=answer_relevancy)
context_rel_chain = RagasEvaluatorChain(metric=context_precision)
context_recall_chain = RagasEvaluatorChain(metric=context_recall)


result_with_truth = result
result_with_truth["ground_truths"] = "Baichuan 2 outperforms in math, code, medical, and legal tasks."

# 获取结果
faithfulness_result = faithfulness_chain(result)
answer_rel_result = answer_rel_chain(result)
context_rel_result = context_rel_chain(result)
context_recall_result = context_recall_chain(result_with_truth)

# 打印四个数字
print("Faithfulness result:", faithfulness_result['faithfulness_score'])
print("Answer relevancy result:", answer_rel_result['answer_relevancy_score'])
print("Context precision result:", context_rel_result['context_precision_score'])
print("Context recall result:", context_recall_result['context_recall_score'])


Faithfulness result: 0.0
Answer relevancy result: 0.9149480200095582
Context precision result: 0.249999999975
Context recall result: 1.0


question 1

Faithfulness result: 0.5

Answer relevancy result: 1.0000000000000009

Context precision result: 0.499999999975

Context recall result: 1.0

question 2

Faithfulness result: 1.0

Answer relevancy result: 0.9866963453736054

Context precision result: 0.99999999995

Context recall result: 1.0

question 3

Faithfulness result: 0.45454545454545453

Answer relevancy result: 0.9947860139442497

Context precision result: 0.6388888888675925

Context recall result: 1.0

1.   Faithfuless

    意义：衡量了生成的答案在给定上下文中的事实一致性。

    步骤：

    1.首先使用LLM来根据问题和答案提取一组语句S。

    2.针对生成的每个语句s，再次使用大模型或验证函数来判断这个语句是否能用上下文中的信息来支撑。

    3.F=V/S

2.   Answer relevancy

    意义：答案与问题的相关程度。不考虑答案的正确性，但是对答案不完整或包含冗余信息的情况进行惩罚。

    步骤：

    1.根据最终答案，利用大模型生成针对该问题的多个潜在的问题。

    2.针对生成的每个潜在问题，利用OpenAI的嵌入模型 text-embedding-ada-002 来计算与原始问题的向量相似度（余弦距离）。




3.   context_relevancy

    意义：检索回的上下文与原始问题之间的相关性，对其中的冗余信息进行惩罚

    1.利用大模型，从给定的context上下文信息中，提取出所有对最终答案直接相关或重要的句子，不改变句子内容。

    2.对答案有用的句子数量 / 上下文中全部句子的数量


In [ ]:
import ragas
import os

# 获取 ragas 包的位置
ragas_path = os.path.dirname(ragas.__file__)
print(f"Ragas package path: {ragas_path}")

# 列出 ragas.metrics.base 模块的内容
import ragas.metrics.base as base
print(dir(base))


Ragas package path: /usr/local/lib/python3.10/dist-packages/ragas
['ABC', 'CallbackManager', 'Dataset', 'Enum', 'EvaluationMode', 'Metric', 'MetricWithLLM', 'RagasEmbeddings', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'abstractmethod', 'annotations', 'dataclass', 'field', 'floor', 'llm_factory', 'make_batches', 't', 'tqdm', 'trace_as_chain_group']


In [13]:
from ragas.metrics.base import Metric
class AnswerConciseness(Metric):
    def __init__(self, batch_size=1):
        super().__init__(batch_size=batch_size)

    @property
    def name(self):
        return "answer_conciseness"

    @property
    def evaluation_mode(self):
        return EvaluationMode.qac  # 使用 'qac' 模式

    def init_model(self):
        pass

    def _score_batch(self, batch, callback_group_name=None, callbacks=None):
        results = []
        for result in batch:
            answer_length = len(result['result'].split())
            score = max(0, 1 - answer_length / 100)  # 假设100个词是一个合理的上限
            results.append({'answer_conciseness_score': score})
        return results

    def __str__(self):
        return "AnswerConciseness"


In [16]:
from ragas.langchain.evalchain import RagasEvaluatorChain, EvaluationMode


# 定义自定义评估链
answer_conciseness_chain = RagasEvaluatorChain(metric=AnswerConciseness(batch_size=1))

result_with_truth = result
result_with_truth["ground_truths"] = ["Baichuan 2 outperforms in math, code, medical, and legal tasks."]



# 获取评估结果

eval_result_conciseness = answer_conciseness_chain(result)

# 打印评估结果

print("答案简洁性:", eval_result_conciseness._score_batch['answer_conciseness_score'])



KeyError: 'result'

In [12]:
from ragas.metrics.base import Metric, EvaluationMode

class AnswerConciseness(Metric):
    def __init__(self, batch_size=1):
        super().__init__(batch_size=batch_size)

    @property
    def name(self):
        return "answer_conciseness"

    @property
    def evaluation_mode(self):
        return EvaluationMode.qac

    def init_model(self):
        pass

    def _score_batch(self, batch, callback_group_name=None, callbacks=None):
        results = []
        for result in batch:
            answer_length = len(result['result'].split())
            score = max(0, 1 - answer_length / 100)  # 假设100个词是一个合理的上限
            results.append({'answer_conciseness_score': score})
        return results

    def __str__(self):
        return "AnswerConciseness"


batch = [result]

# 创建 AnswerConciseness 实例并评分
answer_conciseness = AnswerConciseness()
scores = answer_conciseness._score_batch(batch)
print(scores)

[{'answer_conciseness_score': 0.53}]
